# setup

In [ ]:
run_test=True
DO_TRAIN_VOLS = True
DO_TRAIN_PRCS = True
Provisional=True
MODE = 'tuning'

# run_test=False
# DO_TRAIN_VOLS = False   # 是否运行 VOLS 的 VOL_GRID 训练
# DO_TRAIN_PRCS = False  # 是否运行 PRCS 的 LMSE x VOL_GRID 交叉训练
# Provisional=False
# MODE = 'prod'


epochs=(1,1,1,0,100) if Provisional else (15,500,3,10,100)

cutdate='2025-11-18'
valData=''
use_amp=True
use_amp=False
# valData='2512'
testData=''
testData='new/'

weight=(0.81,1.01,0.03,0)
itr=5
metrics_cnt=6
import torch,sys

if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  mydrive='/content/drive/MyDrive/volrt/'
  checkpoints=mydrive+"checkpoints/"
  source_path=mydrive+"git/Crossformer"
  Provisional=Provisional and run_test
  tables = [ 'volvTSLA.csv', 'volvAAPL.csv'] if Provisional else [ 'volvTSLA.csv', 'volvAAPL.csv', 'volvGOOG.csv','volvNVDA.csv', 'volvMETA.csv', 'volvMSFT.csv', 'volvAMZN.csv', ]
  batch_size=int(torch.cuda.get_device_properties(0).total_memory/1e9+.5)*32
  import os,sys
  if not os.path.exists(source_path):
    %cd $mydrive/git
    !git clone https://github.com/jerronl/Crossformer.git
    %cd $source_path
  else:
    %cd $source_path
    # !git reset --hard HEAD
    # !git pull origin master
  %pip install einops #scikit-learn
  sys.path.append( source_path)
else:
  tables = [ 'volvAAPL.csv']
  testData=''
  mydrive= 'E:/mydoc/git/trade/analyics/'
  batch_size=32
  sys.path.append("G:/git/Crossformer1/")
  itr=1
  %load_ext autoreload
  %autoreload 2
%cd $mydrive


# Params

In [ ]:

# === Hyperparameter Grids (Dictionary Style) ===
# Format: { "arg_name": ([list_of_values], in_vols_bool, in_prcs_bool) }
PARAM_GRID = {
    "lambda_var":  ([0.0004,0.0005],     True,  True),  # 基础方差权重
    "weight_over": ([1.2, 1.5, 3.0],   True,  True),  # 非对称惩罚
    "dist_alpha":  ([1.0, 3.0],   True,  True),  # 软标签锐度
    "lambda_mse":  ([0.5, 1.0, 1.5],   False, True),  # 仅用于 Prcs
    # 未来想加参数直接在这行添加，例如:
    # "dropout":   ([0.1, 0.5],   True,  True),
}



# Funcs

In [ ]:
from cross_exp.exp_crossformer import Exp_crossformer
from data.data_loader import DatasetMTS, data_columns, data_names
from data.data_def import set_cat
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import itertools
import traceback
import matplotlib.gridspec as gridspec
from cross_exp.exp_crossformer import Exp_crossformer
from data.data_loader import DatasetMTS, data_columns, data_names
from data.data_def import set_cat
from utils.tools import init_args, update_args

set_cat(20)
args = init_args()
data_parser = {}

COMMON_SETTINGS = {
    "e_layers": 5,
    "d_model": 512,
    "lradj": "type2",
    "checkpoints": checkpoints,
}


def generate_configs(grid, mode_idx):
    active_keys = []
    values_lists = []
    for key, (vals, in_vols, in_prcs) in grid.items():
        is_active = in_vols if mode_idx == 1 else in_prcs
        if is_active:
            active_keys.append(key)
            values_lists.append(vals)
    configs = []
    for combination in itertools.product(*values_lists):
        cfg = dict(zip(active_keys, combination))
        configs.append(cfg)
    return configs


def apply_config_override(parser_config: dict, ds: str, config_dict: dict) -> dict:
    if ds not in parser_config:
        temp_config = parser_config.copy()
    else:
        temp_config = parser_config[ds].copy()
    temp_config.update(config_dict)
    if ds not in parser_config:
        return temp_config
    else:
        parser_config[ds] = temp_config
        return parser_config


def build_parser(mode, ds, extra_query="", root_override=None):
    root = mydrive + valData if mode == "insample" else mydrive + testData
    if root_override:
        root = root_override
    q = (
        extra_query
        if mode == "insample"
        else (f"date>'#{cutdate}'" + (" and " + extra_query if extra_query else ""))
    )
    ds_config = COMMON_SETTINGS.copy()
    ds_config["root_path"] = root
    ds_config["data_path"] = tables
    ds_config["weight"] = weight[0] if ds == "vols" else weight[1]
    ds_config["data"] = ds
    if q:
        ds_config["query"] = q
    parser = {}
    if ds != "vols":
        parser["vols"] = {"checkpoints": checkpoints}
    parser[ds] = ds_config
    return parser


def build_train_parser(root_path=None):
    if root_path == None:
        root_path = mydrive + valData
    base = {
        "patience": epochs[0],
        "train_epochs": TRAIN_EPOCHS,
        "learning_rate": 0.01,
        "data_split": [0.7, 0.15, 0.15],
        "batch_size": batch_size * 2 // 5,
        "e_layers": 5,
        "d_model": 512,
        "lradj": "type2",
        "checkpoints": checkpoints,
        "root_path": root_path,
        "data_path": tables,
        "profile_mode": False,
        "use_amp": use_amp,
    }
    return {
        "vols": {**base, "data": "vols", "weight": weight[0], "over_weight": weight[2]},
        "prcs": {**base, "data": "prcs", "weight": weight[1], "over_weight": weight[3]},
    }


def get_feature_names(ds_name, in_len):
    cols = data_columns(ds_name)
    return data_names(cols, 1)[0]


def get_trained_epochs(setting_str, ds):
    folder_path = os.path.join(checkpoints, setting_str) + ds
    file_path = os.path.join(folder_path, "checkpoint.pth")
    if os.path.exists(file_path):
        try:
            state = torch.load(file_path, weights_only=False)
            if isinstance(state, dict) and "epoch" in state:
                return state["epoch"]
            else:
                return "N/A"
        except Exception:
            return "Err"
    else:
        return "Not Fnd"


def prepare_preds_trues_for_plot(preds, trues, ycat=0):
    preds = np.asarray(preds)
    trues = np.asarray(trues)
    if ycat > 0:
        reg_dim = preds.shape[1] - ycat
        reg_preds = preds[:, :reg_dim]
        if trues.shape[1] >= reg_dim:
            reg_trues = trues[:, :reg_dim]
        else:
            min_dim = min(trues.shape[1], reg_dim)
            reg_preds = reg_preds[:, :min_dim]
            reg_trues = trues[:, :min_dim]
        return reg_preds, reg_trues
    return preds, trues


def _metrics_text(t, p):
    diff = p - t
    mae = np.mean(np.abs(diff))
    mse = np.mean(diff**2)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs(diff) / (np.abs(t) + 1e-12))
    return f"mae={mae:.4g}\nrmse={rmse:.4g}\nmape={mape:.3g}"


def _metrics_text_prcs_ic_iv(t_iv, p_iv, t_ic, p_ic_label):
    diff = p_iv - t_iv
    mae = np.mean(np.abs(diff))
    mse = np.mean(diff**2)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs(diff) / (np.abs(t_iv) + 1e-12))
    accr = np.nan
    if len(t_ic) > 0 and (np.unique(t_ic).size <= 20) and (~np.isnan(t_ic)).sum() > 0:
        if np.unique(p_ic_label).size <= 20:
            accr = np.mean(
                (p_ic_label.round() == t_ic.round())[
                    ~np.isnan(p_ic_label) & ~np.isnan(t_ic)
                ]
            )
    if not np.isnan(accr):
        return f"accr(IC)={accr:.3f}\nmae(IV)={mae:.4g}\nrmse(IV)={rmse:.4g}"
    else:
        return f"mae(IV)={mae:.4g}\nrmse(IV)={rmse:.4g}\nmape(IV)={mape:.3g}"


def _compute_limits_like_old_regplot(trues_list, preds_list, feat_idx, feat_name):
    left, right = np.inf, -np.inf
    for trues, preds in zip(trues_list, preds_list):
        t = trues[:, feat_idx]
        p = preds[:, feat_idx]
        mask = ~np.isnan(t) & ~np.isnan(p)
        if not mask.any():
            continue
        t, p = t[mask], p[mask]
        if feat_name[:3] not in ["dtm", "pmc"]:
            left = min(left, max(np.min(t), -5), max(np.min(p), -5))
            right = max(right, min(np.max(t), 5), min(np.max(p), 5))
        else:
            left = min(left, np.min(t), np.min(p))
            right = max(right, np.max(t), np.max(p))
    if not np.isfinite(left) or not np.isfinite(right) or left == right:
        left, right = -1, 1
    return left, right


def plot_residual_heatmap_family(
    preds_raw,
    trues_raw,
    feature_names,
    title,
    bins=80,
    ycat=0,
    cols=4,
    figsize_per_cell=4,
):
    preds, trues = prepare_preds_trues_for_plot(preds_raw, trues_raw, ycat=ycat)
    out_dim = trues.shape[1]
    max_plot_dim = len(feature_names)
    effective_dim = min(out_dim, max_plot_dim)
    rows = int(np.ceil(effective_dim / cols))
    fig, axes = plt.subplots(
        rows,
        cols,
        figsize=(figsize_per_cell * cols, figsize_per_cell * rows),
        squeeze=False,
        constrained_layout=True,
    )
    for i in range(effective_dim):
        r, c = divmod(i, cols)
        ax = axes[r, c]
        fname = feature_names[i]
        t = trues[:, i]
        p = preds[:, i]
        mask = ~np.isnan(t) & ~np.isnan(p)
        t, p = t[mask], p[mask]
        if len(t) == 0:
            ax.set_title(fname)
            ax.axis("off")
            continue
        if ycat > 0 and i == 0:
            p_cls = preds[mask, 4]
            metrics_text = _metrics_text_prcs_ic_iv(t, p, t, p_cls)
        else:
            metrics_text = _metrics_text(t, p)
        left, right = _compute_limits_like_old_regplot(
            [trues_raw], [preds_raw], i, fname
        )
        ax.hist2d(t, p, bins=bins, range=[[left, right], [left, right]], norm="log")
        ax.plot([left, right], [left, right], "w--", alpha=0.45)
        ax.set_xlim(left, right)
        ax.set_ylim(left, right)
        ax.set_xlabel("True")
        ax.set_ylabel("Pred")
        ax.set_title(fname, fontsize=12)
        ax.text(
            0.02,
            0.98,
            metrics_text,
            transform=ax.transAxes,
            va="top",
            ha="left",
            fontsize=9,
            bbox=dict(boxstyle="round,pad=0.25", fc="black", ec="none", alpha=0.35),
            color="white",
            zorder=3,
        )
    for i in range(effective_dim, rows * cols):
        axes.flatten()[i].axis("off")
    if title:
        fig.suptitle(title, fontsize=14)
    plt.show()
    plt.close(fig)


def plot_multi_stock_residuals(
    all_preds_raw,
    all_trues_raw,
    stock_labels,
    feature_names,
    ycat=0,
    title="",
    bins=80,
    cols=4,
    figsize_per_cell=4,
):
    if not all_trues_raw:
        return
    out_dim = all_trues_raw[0].shape[1]
    max_plot_dim = len(feature_names)
    effective_dim = min(out_dim, max_plot_dim)
    rows = int(np.ceil(effective_dim / cols))
    fig = plt.figure(figsize=(figsize_per_cell * cols, figsize_per_cell * rows))
    outer = gridspec.GridSpec(rows, cols, wspace=0.25, hspace=0.35)
    cmap = plt.get_cmap("tab10") if len(stock_labels) <= 10 else plt.get_cmap("tab20")
    colors = [cmap(i) for i in range(len(stock_labels))]
    all_preds_reg, all_trues_reg = [], []
    for preds_raw, trues_raw in zip(all_preds_raw, all_trues_raw):
        p_reg, t_reg = prepare_preds_trues_for_plot(preds_raw, trues_raw, ycat=ycat)
        all_preds_reg.append(p_reg)
        all_trues_reg.append(t_reg)
    for i in range(effective_dim):
        r, c = divmod(i, cols)
        ax_main = fig.add_subplot(outer[r, c])
        ax_hist_bg = ax_main.twinx()
        fname = feature_names[i]
        combined_t, combined_p = all_trues_reg, all_preds_reg
        left, right = _compute_limits_like_old_regplot(combined_t, combined_p, i, fname)
        max_hist = 0
        for j, (preds_reg, trues_reg) in enumerate(zip(all_preds_reg, all_trues_reg)):
            t, p = trues_reg[:, i], preds_reg[:, i]
            mask = ~np.isnan(t) & ~np.isnan(p)
            t, p = t[mask], p[mask]
            ax_main.scatter(
                t,
                p,
                s=5,
                alpha=0.4,
                label=stock_labels[j],
                color=colors[j % len(colors)],
                zorder=2,
            )
            if len(t) > 1:
                h, _, _ = ax_hist_bg.hist(
                    t,
                    bins=bins,
                    density=True,
                    alpha=0.15,
                    color=colors[j % len(colors)],
                    zorder=1,
                )
                max_hist = max(max_hist, np.max(h))
        all_t = np.concatenate([t[:, i] for t in combined_t])
        all_p = np.concatenate([p[:, i] for p in combined_p])
        mask = ~np.isnan(all_t) & ~np.isnan(all_p)
        t_all, p_all = all_t[mask], all_p[mask]
        if len(t_all) == 0:
            ax_main.set_title(fname)
            ax_main.axis("off")
            continue
        if ycat > 0 and i == 0:
            all_p_cls = np.concatenate([p[:, 4] for p in combined_p])
            p_cls_all = all_p_cls[mask]
            metrics_text = _metrics_text_prcs_ic_iv(t_all, p_all, t_all, p_cls_all)
        else:
            metrics_text = _metrics_text(t_all, p_all)
        ax_hist_bg.tick_params(
            axis="y", left=False, right=False, labelleft=False, labelright=False
        )
        ax_hist_bg.set_ylim(0, max_hist * 3.0 if max_hist > 0 else 1)
        ax_hist_bg.set_ylabel("")
        ax_main.plot([left, right], [left, right], "k--", alpha=0.45)
        ax_main.set_xlim(left, right)
        ax_main.set_ylim(left, right)
        ax_main.set_title(fname, fontsize=10)
        ax_main.text(
            0.02,
            0.98,
            metrics_text,
            transform=ax_main.transAxes,
            va="top",
            ha="left",
            fontsize=9,
            bbox=dict(boxstyle="round,pad=0.25", fc="black", ec="none", alpha=0.35),
            color="white",
            zorder=3,
        )
        if i == effective_dim - 1:
            ax_main.legend(fontsize=7, loc="lower right", framealpha=0.5)
    for i in range(effective_dim, rows * cols):
        fig.add_subplot(outer[i]).axis("off")
    if title:
        fig.suptitle(title, fontsize=14)
        plt.tight_layout(rect=[0, 0, 1, 0.97])
    else:
        plt.tight_layout()
    plt.show()
    plt.close()


def plot_prcs_distribution_detailed(
    results_list,
    model_idx,
    ycat,
    dep_var,
    tables_list,
    cols=4,
    title_prefix="PRCS Distribution",
):
    if not results_list or ycat == 0:
        print("No PRCS results or ycat=0, skipping detailed distribution plot.")
        return
    n_categories = ycat
    x = np.arange(1, n_categories + 1)
    n_tables = len(results_list)
    outer_rows = (n_tables - 1) // cols + 1
    fig = plt.figure(figsize=(16, 4 * outer_rows))
    outer = gridspec.GridSpec(outer_rows, cols, wspace=0.3, hspace=0.45)
    ls = ["-", "--", ":"]
    print(f"\n--- Plotting {title_prefix} for Model m{model_idx} ---")
    for j, result in enumerate(results_list):
        preds_raw, trues_raw, _ = result
        preds = np.asarray(preds_raw)
        trues = np.asarray(trues_raw)
        probs = preds[:, -n_categories:]
        true_labels = np.round(trues[:, 0]).astype(int)
        valid_mask = (
            (~np.isnan(true_labels))
            & (true_labels >= 1)
            & (true_labels <= n_categories)
        )
        if np.sum(valid_mask) == 0:
            continue
        probs = probs[valid_mask]
        true_labels = true_labels[valid_mask]
        if np.size(probs) > 0 and (np.max(probs) > 1.05 or np.min(probs) < -0.05):
            exp_probs = np.exp(probs - np.max(probs, axis=1, keepdims=True))
            probs = exp_probs / (np.sum(exp_probs, axis=1, keepdims=True) + 1e-12)
        df = pd.DataFrame(probs, columns=[str(i) for i in range(1, n_categories + 1)])
        df["True_Label"] = true_labels
        summed_probs = df.groupby("True_Label").mean().T
        xi = [int(x) for x in summed_probs.index]
        third = n_categories // 3 + 1
        r_outer = j // cols
        c_outer = j % cols
        sub_gs = outer[r_outer, c_outer].subgridspec(2, 1, hspace=0.0)
        valid_true_labels = sorted(
            [c for c in summed_probs.columns if 1 <= c <= n_categories]
        )
        for k in (0, 1):
            ax = fig.add_subplot(sub_gs[k, 0])
            hist_bins = np.arange(1, n_categories + 2) - 0.5
            ax.hist(
                true_labels,
                bins=hist_bins,
                alpha=0.2,
                density=True,
                color="gray",
                label="True",
                zorder=1,
            )
            ci = 0
            for m in range(k, len(valid_true_labels), 2):
                col = valid_true_labels[m]
                ax.plot(
                    xi,
                    summed_probs[col],
                    label=f"cat {col}",
                    linestyle=ls[col // third % 3],
                    color=f"C{ci % 10}",
                    alpha=0.7,
                    zorder=2,
                )
                pred_prob_at_true_class = summed_probs[col][str(col)]
                ax.text(
                    col,
                    pred_prob_at_true_class,
                    f"{col}",
                    fontsize=8,
                    va="bottom",
                    ha="center",
                    color=f"C{(ci) % 10}",
                    zorder=3,
                )
                ci += 1
            ax.set_title(
                (
                    f'{tables_list[j]} - {"Odd" if k == 0 else "Even"})'
                    if k == 0
                    else ""
                ),
                fontsize=10,
            )
            ax.set_ylim(0.0, 0.2)
            ax.set_xlim(0.5, n_categories + 1.5)
            ax.set_xticks(x)
            ax.set_xticklabels([str(i) for i in x], rotation=-45, fontsize=8)
            ax.legend(
                loc="center left",
                bbox_to_anchor=(1.05, 0.5),
                fontsize=7,
                framealpha=0.6,
            )
            ax.grid(axis="y", linestyle="--", alpha=0.3)
            if k == 0:
                plt.setp(ax.get_xticklabels(), visible=False)
                ax.set_xlabel("")
            else:
                ax.set_xlabel("Predicted Class Index", fontsize=9)
    fig.suptitle(f"{title_prefix} - Model m{model_idx}", fontsize=14)
    plt.tight_layout(rect=[0, 0.03, 1, 0.97])
    plt.show()
    plt.close()


def plot_metric_summary(summary_records):
    if not summary_records:
        return
    df = pd.DataFrame(summary_records)
    metrics = ["mae", "mse", "rmse", "mape", "accr"]
    metrics = [m for m in metrics if m in df.columns]
    if not metrics:
        return
    cols = 2
    rows = int(np.ceil(len(metrics) / cols))
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    axes = axes.flatten()
    cmap = plt.get_cmap("tab10")
    for i, metric in enumerate(metrics):
        ax = axes[i]
        if "horizon" in df.columns:
            for idx_val in sorted(df["idx"].unique()):
                sub = df[df["idx"] == idx_val].sort_values("horizon")
                ax.plot(
                    sub["horizon"],
                    sub[metric],
                    marker="o",
                    label=f"Cfg {idx_val}",
                    linewidth=2,
                )
            ax.set_xlabel("Horizon")
        else:
            ax.plot(df["idx"], df[metric], marker="o", linestyle="-", color=cmap(i))
            ax.set_xlabel("Config Index")
            ax.set_xticks(df["idx"].unique())
        ax.set_title(metric.upper())
        ax.set_ylabel(metric)
        ax.grid(True, linestyle="--", alpha=0.5)
        if "horizon" in df.columns:
            ax.legend()
    for i in range(len(metrics), len(axes)):
        axes[i].axis("off")
    plt.suptitle("Metric Comparison Summary", fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.97])
    plt.show()
    plt.close()


def generate_final_summary(summary_records):
    if not summary_records:
        print("No records to summarize.")
        return
    df = pd.DataFrame(summary_records)
    fixed_info_cols = ["dataset", "idx", "horizon", "epochs"]
    metric_cols = ["mae", "mse", "rmse", "mape", "mdst", "accr"]
    all_cols = df.columns.tolist()
    param_cols = [
        c for c in all_cols if c not in fixed_info_cols and c not in metric_cols
    ]
    final_order = []
    for c in fixed_info_cols:
        if c in df.columns:
            final_order.append(c)
    final_order += sorted(param_cols)
    final_order += [c for c in metric_cols if c in df.columns]
    print(df[final_order].to_string(float_format=lambda x: "{:.4g}".format(x)))


def run_test_one_setting(
    exp,
    setting_str,
    ds,
    dep_var,
    ycat,
    title_prefix,
    by_stock=False,
    stock_tables=None,
    plot_individual_heatmaps=True,
):
    metrics_list, all_preds, all_trues = [], [], []

    def _plot_results(p_raw, t_raw, current_title):
        if not plot_individual_heatmaps:
            return
        print(f"\n--- Residual Heatmap: {current_title} ---")
        plot_residual_heatmap_family(
            p_raw, t_raw, dep_var, title=current_title, ycat=ycat
        )

    if by_stock:
        for tb in stock_tables:
            preds, trues, m = exp.test(
                setting_str, ds, True, data_path=[tb], inverse=True
            )
            metrics_list.append(m)
            all_preds.append(preds)
            all_trues.append(trues)
            _plot_results(preds, trues, f"{title_prefix} {tb}")
        return metrics_list, all_preds, all_trues
    else:
        preds, trues, m = exp.test(setting_str, ds, True, inverse=True)
        metrics_list.append(m)
        _plot_results(preds, trues, title_prefix)
        return metrics_list, None, None


def train_unified(target_ds: str, best_idx: int = 0):
    global args, data_parser, MODE, epochs
    if target_ds == "vols":
        configs = VOLS_CONFIGS
    elif target_ds == "prcs":
        configs = PRCS_CONFIGS
    else:
        return
    base_parser = build_train_parser()
    if not configs:
        print(f"No {target_ds.upper()} configs generated. Skipping training.")
        return
    start_lr = base_parser[target_ds]["learning_rate"]
    print(
        f"\n=== Starting {target_ds.upper()} Training (Mode: {MODE}, Total configs: {len(configs)}) ==="
    )
    if MODE == "prod":
        if best_idx < 0 or best_idx >= len(configs):
            print(f"Error: Invalid best_idx ({best_idx}). Aborting PROD run.")
            return
        cfg_idx = best_idx
        for run_itr in range(args.itr):
            if run_itr > 0:
                DatasetMTS.clear()
                print(f"\n--- Production Run {run_itr+1}/{args.itr} Started ---")
            cfg = configs[cfg_idx]
            current_lr = start_lr
            current_parser = {k: v.copy() for k, v in base_parser.items()}
            current_parser[target_ds]["learning_rate"] = current_lr
            current_parser[target_ds] = apply_config_override(
                current_parser[target_ds], target_ds, cfg
            )
            setting_str = update_args(
                args, current_parser, itr=run_itr, arg_set=target_ds
            )
            exp = Exp_crossformer(args)
            cfg_display = ", ".join([f"{k}={v}" for k, v in cfg.items()])
            print(f"[{target_ds.upper()} | Run {run_itr+1}] Setting: {setting_str}")
            print(f"   LR={current_lr:.6f} | Params: {{{cfg_display}}}")
            try:
                exp.train(setting_str, target_ds)
            except Exception as e:
                print(f"--- FAILED TRAINING {target_ds} index {cfg_idx} --- Error: {e}")
                traceback.print_exc()
    elif MODE == "tuning":
        num_cycles = epochs[2]
        for cycle in range(num_cycles):
            current_lr = start_lr / (3.0**cycle)
            base_parser[target_ds]["learning_rate"] = current_lr
            for idx in active_grid_indices:
                if idx >= len(configs):
                    break
                cfg = configs[idx]
                current_parser = {k: v.copy() for k, v in base_parser.items()}
                current_parser[target_ds] = apply_config_override(
                    current_parser[target_ds], target_ds, cfg
                )
                setting_str = update_args(
                    args, current_parser, itr=0, arg_set=target_ds
                )
                exp = Exp_crossformer(args)
                cfg_display = ", ".join([f"{k}={v}" for k, v in cfg.items()])
                print(
                    f"[{target_ds.upper()} | Cyc {cycle+1} | Idx {idx}] Setting: {setting_str}"
                )
                print(f"   LR={current_lr:.6f} | Params: {{{cfg_display}}}")
                try:
                    exp.train(setting_str, target_ds)
                except Exception as e:
                    print(f"--- FAILED TRAINING {target_ds} index {idx} --- Error: {e}")
                    traceback.print_exc()


def test_unified(
    mode: str,
    target_ds: str,
    by_stock: bool = True,
    plot_residuals: bool = True,
    plot_stock: bool = False,
):
    global args
    if target_ds == "vols":
        configs = VOLS_CONFIGS
        ycat = 0
    elif target_ds == "prcs":
        configs = PRCS_CONFIGS
        ycat = data_columns(target_ds).get("ycat", 0)
    else:
        return
    dep_var = get_feature_names(target_ds, getattr(args, "seq_len", 1))
    summary_data = []
    if not configs:
        return
    print(
        f"\n=== Starting {target_ds.upper()} Testing ({mode}) ({len(configs)} configs) ==="
    )
    for idx in active_grid_indices:
        if idx >= len(configs):
            break
        cfg = configs[idx]
        base_parser = build_parser(mode, target_ds)
        ds_config = apply_config_override(base_parser[target_ds].copy(), target_ds, cfg)
        ds_config["data"] = target_ds
        base_parser[target_ds] = ds_config
        setting_str = update_args(args, base_parser, itr=0, arg_set=target_ds)
        trained_epochs = get_trained_epochs(setting_str, target_ds)
        preds_fam = None
        trues_fam = None
        try:
            exp = Exp_crossformer(args)
            cfg_str = ", ".join([f"{k}={v}" for k, v in cfg.items()])
            title = f"{target_ds} {mode} [Idx {idx}] {cfg_str}"
            m_list, preds_fam, trues_fam = run_test_one_setting(
                exp,
                setting_str,
                target_ds,
                dep_var,
                ycat,
                title,
                by_stock=by_stock,
                stock_tables=tables,
                plot_individual_heatmaps=plot_stock,
            )
            if by_stock and preds_fam is not None and plot_residuals:
                print(f"\n--- AGGREGATED RESIDUAL PLOT: {title} ---")
                plot_multi_stock_residuals(
                    preds_fam,
                    trues_fam,
                    tables,
                    dep_var,
                    ycat=ycat,
                    title=f"AGGREGATED {title}",
                )
                if target_ds == "prcs" and ycat > 0:
                    results_list = list(zip(preds_fam, trues_fam, m_list))
                    print(f"\n--- DETAILED PRCS DISTRIBUTION: {title} ---")
                    plot_prcs_distribution_detailed(
                        results_list=results_list,
                        model_idx=idx,
                        ycat=ycat,
                        dep_var=dep_var,
                        tables_list=tables,
                        cols=4,
                        title_prefix=f"AGG {title}",
                    )
            avg_metrics = np.mean(np.array(m_list), axis=0)
            rec = {
                "dataset": target_ds,
                "idx": idx,
                "epochs": trained_epochs,
                **cfg,
                "mae": avg_metrics[0],
                "mse": avg_metrics[1],
                "rmse": avg_metrics[2],
                "mape": avg_metrics[3],
                "mdst": avg_metrics[4],
                "accr": avg_metrics[5],
            }
            summary_data.append(rec)
        except Exception as e:
            print(f"--- FAILED TESTING {target_ds} idx {idx} --- Error: {e}")
            traceback.print_exc()
    generate_final_summary(summary_data)
    if len(summary_data) > 1:
        print("\n--- Plotting Metric Comparisons ---")
        plot_metric_summary(summary_data)
    return summary_data


def test_vols_per_horizon(
    mode: str, plot_residuals: bool = True, best_idx: int = 0, plot_stock: bool = False
):
    global args, VOLS_CONFIGS, MODE
    ds = "vols"
    horizons = list(range(5, 0, -1))
    dep_var = get_feature_names(ds, getattr(args, "seq_len", 1))
    summary_data = []
    print(f"\n--- Running {mode} VOLS Per-Horizon Test (Horizons: {horizons}) ---")
    if MODE == "prod":
        configs_to_run = [VOLS_CONFIGS[best_idx]]
        indices_to_run = [best_idx]
    else:
        configs_to_run = VOLS_CONFIGS
        indices_to_run = active_grid_indices
    for p_idx in indices_to_run:
        if p_idx >= len(configs_to_run):
            break
        cfg_idx = p_idx
        cfg = configs_to_run[p_idx] if MODE == "tuning" else configs_to_run[0]
        cfg_display = ", ".join([f"{k}={v}" for k, v in cfg.items()])
        print(f"\n[VOLS Horizon Test | Idx {cfg_idx}] Params: {{{cfg_display}}}")
        for h_len in horizons:
            base_parser = build_parser(mode, ds)
            parser = {k: v.copy() for k, v in base_parser.items()}
            parser[ds] = apply_config_override(parser[ds].copy(), ds, cfg)
            parser[ds]["data"] = ds
            hq = f"floor(horizon)=={h_len}"
            existing_query = parser[ds].get("query", "")
            parser[ds]["query"] = (existing_query + " and " + hq).lstrip(" and ")
            setting_str = update_args(args, parser, 0, ds)
            trained_epochs = get_trained_epochs(setting_str, ds)
            try:
                title_prefix = f"VOLS {mode} H={h_len} [Idx {cfg_idx}]"
                exp = Exp_crossformer(args)
                m_list, current_preds_fam, current_trues_fam = run_test_one_setting(
                    exp,
                    setting_str,
                    ds,
                    dep_var,
                    ycat=0,
                    title_prefix=title_prefix,
                    by_stock=True,
                    stock_tables=tables,
                    plot_individual_heatmaps=plot_stock,
                )
                if m_list:
                    avg_metrics = np.mean(np.array(m_list), axis=0)
                    rec = {
                        "dataset": ds,
                        "idx": cfg_idx,
                        "horizon": h_len,
                        "epochs": trained_epochs,
                        **cfg,
                        "mae": avg_metrics[0],
                        "mse": avg_metrics[1],
                        "rmse": avg_metrics[2],
                        "mape": avg_metrics[3],
                        "mdst": avg_metrics[4],
                        "accr": avg_metrics[5],
                    }
                    summary_data.append(rec)
                if plot_residuals and current_preds_fam is not None:
                    print(f"\n--- AGGREGATED RESIDUAL PLOT: {title_prefix} ---")
                    plot_multi_stock_residuals(
                        current_preds_fam,
                        current_trues_fam,
                        tables,
                        dep_var,
                        ycat=0,
                        title=f"AGGREGATED {title_prefix}",
                    )
            except Exception as e:
                print(
                    f"--- FAILED Horizon Test H={h_len}, idx {cfg_idx} --- Error: {e}"
                )
                traceback.print_exc()
    generate_final_summary(summary_data)
    if len(summary_data) > 1:
        print("\n--- Plotting Metric Comparisons (Horizon) ---")
        plot_metric_summary(summary_data)
    return summary_data


In [ ]:
VOLS_CONFIGS = generate_configs(PARAM_GRID, 1)
PRCS_CONFIGS = generate_configs(PARAM_GRID, 2)

if Provisional:
    VOLS_CONFIGS = VOLS_CONFIGS[:2]
    PRCS_CONFIGS = PRCS_CONFIGS[:2]

total_configs = (len(VOLS_CONFIGS) if MODE == "tuning" else len(PRCS_CONFIGS))
start_idx = epochs[3]
end_idx = min(start_idx + epochs[4], total_configs)
active_grid_indices = range(start_idx, end_idx)

print("--- Config Preview (Active Slice) ---")
target_configs = VOLS_CONFIGS if len(VOLS_CONFIGS) > 0 else PRCS_CONFIGS
for i in active_grid_indices:
    if i < len(target_configs):
        print(f"  [{i}] {target_configs[i]}")

if MODE == "tuning":
    TRAIN_EPOCHS = min(30, epochs[1])
else:
    TRAIN_EPOCHS = epochs[1]

# train

In [ ]:
if run_test == False:

    if DO_TRAIN_VOLS:
        print("\n--- [Train VOLS with VOL_GRID] ---")
        train_unified('vols')

    if DO_TRAIN_PRCS:
        print("\n--- [Train PRCS with LMSE x VOL_GRID] ---")
        train_unified('prcs')

In [ ]:
if not run_test:
    from google.colab import runtime
    runtime.unassign()

In [ ]:
if not run_test:
    from cross_exp.exp_crossformer import Exp_crossformer
    from data.data_loader import DatasetMTS
    data_parser = {
        "vols": {
            'data':'vols',
            "patience":epochs[0],
            "train_epochs":epochs[1],
            'learning_rate':0.01,
            'data_split':[0.7,0.15,0.15],
            'batch_size':batch_size*2//5,
            'e_layers':5,
            'd_model':512,
            'lradj':'type2',
            "checkpoints":checkpoints,
            'root_path':mydrive+valData,
            'data_path':tables,
            'weight':weight[0],
            'over_weight':weight[2],
            'profile_mode':False,
            'use_amp':use_amp
        },
        "prcs":{
            'data':'prcs',
            'weight':weight[1],
            'over_weight':weight[3],
        }
        }
    s=epochs[3]
    for _ in range(epochs[2]):
        data_parser["vols"]["learning_rate"]=.01
        for i in range(epochs[2]):
            for ii in range(s,itr):
                # setting record of experiments
                # data_parser["vols"]['weight']=weight[0]
                setting=update_args(args,data_parser,ii)
                DatasetMTS.clear()
                # drive.mount('/content/drive/',force_remount=True)
                # !google-drive-ocamlfuse -cc # 注释掉这个bug行

                exp = Exp_crossformer(args)  # set experiments
                print(f"start training :lr={data_parser["vols"]["learning_rate"]} {setting}")
                exp.train(setting, "vols")

                print(f">>>>>>>testing : {setting}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
                preds, trues,_ = exp.test(setting, 'vols', True, data_path=[tables[-1]], inverse=True)
                print(preds.shape, trues.shape)

                # data_parser["vols"]['weight']=weight[1]
                setting=update_args(args,data_parser,ii,'prcs')
                DatasetMTS.clear()
                exp = Exp_crossformer(args)  # set experiments
                print(f"start training :lr={data_parser["vols"]["learning_rate"]} {setting}")
                exp.train(setting, "prcs")
                preds, trues,_ = exp.test(setting, 'prcs', True, data_path=[tables[-1]], inverse=True)
                print(preds.shape, trues.shape)
            data_parser["vols"]["learning_rate"]/=3.
            s=0


# test
##  PART A — In-sample (test split of same source)
Subparts:
1. vols per-stock heatmap  
2. prcs per-stock heatmap + avg prob by true class  
3. vols per-horizon heatmap


In [ ]:

if run_test:
            test_unified("insample", "vols")

## In-sample - Prcs Per-Stock

Subparts:
1. vols per-stock heatmap  
2. prcs per-stock heatmap + avg prob by true class  
3. vols per-horizon heatmap


In [ ]:
if run_test:

           test_unified("insample", "prcs")


## In-sample - Horizon


In [ ]:
if run_test:
        test_vols_per_horizon(mode="insample")

# PART B — Out-of-sample (testData, cutdate+)
## OOS - Vols Per-Stock

In [ ]:
if run_test:

        test_unified("oos", "vols")

## hlc

In [ ]:
if run_test:
        test_unified("oos", "prcs")

## horizon


In [ ]:
if run_test:

        test_vols_per_horizon(mode="oos")

# end

In [ ]:
plt.show()
from google.colab import runtime
runtime.unassign()